In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import (
    cluster, datasets,
    decomposition, ensemble, manifold,
    random_projection, preprocessing)
import missingno as msno
from plot import plot
from sklearn.impute import SimpleImputer
from math import e
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn import (
    cluster, datasets,
    decomposition, ensemble, manifold,
    random_projection, preprocessing)
from sklearn.model_selection import RandomizedSearchCV
from matplotlib import cm
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from random import randint

from clean_data import datapipeline


def import_rf_nhanes(model=True):
    clean_nhanes = datapipeline()
    target, features, df = clean_nhanes.create_rf_X_y(model=model)
    # target = target['HDL_OVER_TCHOL']  # .apply(lambda x: np.log(x+1))
    return target, features, df

In [3]:
def run_random_forest_c(features, target, oversample=False, ts=.25):
    # weights = {1:1, 2:16, 3:43, 4:60, 5:585}
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, stratify=target, test_size=ts, random_state=10)
    print(len(y_train))
    if oversample:
        ros = RandomOverSampler(random_state=42)
        X_train, y_train = ros.fit_resample(X_train, y_train)
    # rfc = RandomForestClassifier( class_weight='balanced_subsample')
    rfc = RandomForestClassifier(random_state=32, n_estimators=2000,
                                 min_samples_split=15, min_samples_leaf=69,
                                 class_weight='balanced_subsample')
    rfc.fit(X_train, y_train)
    # print("\n score:", rfc.score(X_test, y_test))
    y_predict = rfc.predict(X_test)

    # y_predict_proba = rfc.predict_proba(X_test)[:, 1]
    # y_predict = y_predict_proba > .475

    # plt.hist(y_predict_proba)
    # plt.show()
    # # breakpoint()
    # f1 = cross_val_score(rfc, X_test, y_test,
    #                      scoring='f1',
    #                      cv=6, n_jobs=-1)
    # print("\n cross val f1", f1)
    print("\n confusion matrix:\n", confusion_matrix(y_test, y_predict))
    # tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    # print(tn, fp, fn, tp)
    print("\n f1 score ", f1_score(y_test, y_predict, average='weighted'))
    return rfc

In [4]:
target, features, df = import_rf_nhanes(model=False)

In [5]:
total_df = features.copy()
    
target['CHOL_RISK'] = 0
target.loc[target['HDL_OVER_TCHOL'] > 3.5, 'CHOL_RISK'] = 1
target.loc[target['HDL_OVER_TCHOL'] > 5.0, 'CHOL_RISK'] = 2
target = target.set_index('SEQN')

In [6]:
drop_cols = ['LBDHDDSI', 'LBDTCSI', 'LBDHDD', 'LBXTC']
for i in drop_cols:
    df.drop(i, axis=1, inplace=True)
for i in df.columns:
    if type(df.loc[0, i]) == str:
        df.drop(i, axis=1, inplace=True)
df = df.set_index('SEQN')

In [7]:
rf = run_random_forest_c(features, target['CHOL_RISK'], oversample=True)

3953

 confusion matrix:
 [[398 180 101]
 [156 161 135]
 [ 38  66  83]]

 f1 score  0.49774291926246145


In [22]:
type(target['CHOL_RISK'].to_numpy())

numpy.ndarray

In [21]:
estimator = rf.estimators_[0]
from sklearn.tree import export_graphviz
print(list(features.columns))
# Export as dot file
print(export_graphviz(estimator, out_file='tree_limited23.dot', feature_names=list(features.columns),
            class_names = target['CHOL_RISK'].to_numpy(), rounded=True, proportion=False, 
            precision=2, filled=True))

['BMXWAIST', 'BMXBMI', 'BMXARMC', 'RIAGENDR', 'RIDAGEYR', 'WHD140', 'BPXDI1', 'WHQ150', 'BMXHT', 'BMXLEG', 'DR1TCHOL', 'BPXSY1', 'DR1TPFAT', 'BMXARML', 'PHAFSTHRMN', 'FITNESS_SCORE_MIN', 'SUMTEETH']


TypeError: can only concatenate str (not "numpy.int64") to str

Note: you may need to restart the kernel to use updated packages.


In [10]:
!dot -Tpng tree_limited.dot -o tree_limited.png -Gdpi=600

zsh:1: command not found: dot


In [11]:
from IPython.display import Image
Image(filename = 'tree_limited.png')

FileNotFoundError: [Errno 2] No such file or directory: 'tree_limited.png'